In [6]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, KFold

TRAIN_PATH = '../../res/ftr/training_data.csv'
TEST_PATH = '../../res/ftr/test_data.csv'

pd.set_option('display.max_columns', 30)

training_set = pd.read_csv(TRAIN_PATH)
evaluation_set = pd.read_csv(TEST_PATH)

training_set.drop(columns=['Unnamed: 0'], inplace=True)
evaluation_set.drop(columns=['Unnamed: 0'], inplace=True)

evaluation_set

,id,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
0,4941,29.0,3.0,NaN,4.0,300.0,NaN,NaN,15.906,0,0,0,0,0
1,51775,NaN,1.0,1.0,1.0,67.0,67.0,113851.0,16.732,0,0,0,0,0
2,115253,0.0,2.0,1.0,2.0,87.0,100.0,23620.0,16.585,0,0,0,0,1
3,299321,2.0,2.0,2.0,2.0,86.0,86.0,129347.0,16.527,0,0,0,0,0
4,173570,10.0,2.0,1.0,1.0,80.0,76.0,57125.0,15.932,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,75094,20.0,4.0,3.0,3.0,291.0,NaN,275741.0,16.697,0,0,0,1,1
59996,171847,10.0,3.0,1.0,2.0,71.0,87.0,57474.0,17.093,0,0,0,1,1
59997,138313,5.0,3.0,2.0,2.0,102.0,NaN,72224.0,16.406,0,0,0,1,1
59998,271268,0.0,2.0,1.0,2.0,130.0,144.0,83960.0,17.095,0,0,0,1,1


In [7]:
'''a = 0
b = int(training_set.shape[0]*0.8)
c = training_set.shape[0]

test_set = training_set.iloc[b:c, :]
training_set = training_set.iloc[a:b, :]'''

training_set_X = training_set.drop(columns=['precio']).values
training_set_Y = training_set['precio'].values

regressor = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.08,
                             gamma=0, subsample=0.75, colsample_bytree=1, max_depth=7)

regressor.fit(training_set_X, training_set_Y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.08, max_delta_step=0,
             max_depth=7, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.75, verbosity=1)

In [8]:
evaluation_set_prediction = regressor.predict(evaluation_set.values)

In [9]:
estimated_prices = pd.DataFrame()
estimated_prices['id'] = evaluation_set['id']
estimated_prices['target'] = evaluation_set_prediction
estimated_prices

estimated_prices.to_csv('../../res/reg/estimated_prices.csv', index=False, header=True)

In [10]:
estimated_prices

,id,target
0,4941,4.814786e+06
1,51775,1.174389e+06
2,115253,2.680676e+06
3,299321,1.272529e+06
4,173570,6.316535e+05
...,...,...
59995,75094,4.999529e+06
59996,171847,7.019391e+05
59997,138313,1.354532e+06
59998,271268,1.703896e+06
